<a href="https://colab.research.google.com/github/pafernannapi18/GithubIntro/blob/main/18Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Problem 1] Cross Validation


In [ ]:
# numpy and pandas for manipulation 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns

# sklearn reprocessing for dealing with categorical variables 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder
from sklearn.metrics import  accuracy_score
from sklearn.model_selection import  KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression  

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/application_train.csv")
df = df.select_dtypes('number')

In [ ]:
# cleaning the dataset by filling the empty data(null)
cleaned_df = df.fillna(0)

In [ ]:
#get only existing data with no missing values 
cleaned_df = cleaned_df[cleaned_df.columns[~cleaned_df.isnull().all()]]

In [ ]:
# Separated them into variables
X = cleaned_df.drop(['TARGET'], axis=1)
y = cleaned_df['TARGET']

X = X.to_numpy()
kf = KFold(n_splits = 2)

In [ ]:
for train_index, test_index in kf.split(X):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

TRAIN: [153756 153757 153758 ... 307508 307509 307510] TEST: [     0      1      2 ... 153753 153754 153755]
TRAIN: [     0      1      2 ... 153753 153754 153755] TEST: [153756 153757 153758 ... 307508 307509 307510]


In [ ]:
# Standardizing the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

## [Problem 2] Grid search

In [ ]:
# checking which model and params are best 
model_params = {'random_forest':{'model': RandomForestClassifier(),'params': {'n_estimators': [1, 5,10]}},
        'logic_regression':{'model': LogisticRegression(solver="liblinear", multi_class = "auto"),'params': {'C': [1,5,10]}}}
# defining an arraay to store the scores
scores = []
for model_name, mp in model_params.items():
  clf = GridSearchCV(mp['model'], mp['params'], return_train_score= False)
  clf.fit(X_train_trans, y_train)
  scores.append({
      'model': model_name, 
      'best_score': clf.best_score_,
      'best_params': clf.best_params_})
best_model_params = pd.DataFrame(scores, columns= ['model', 'best_score', 'best_params'])
best_model_params

,model,best_score,best_params
0,random_forest,0.915782,{'n_estimators': 10}
1,logic_regression,0.918364,{'C': 1}
